In [28]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys


driver = webdriver.Chrome('/Users/adriaferrer/IDIADA_Repos/Moneyball/01_get_data/chromedriver')

username = 'minandaandcastolo@gmail.com'
password = 'wholetthedogs0ut'

# Open the site and click to go to login
driver.get('https://biwenger.as.com/login')
driver.find_element_by_css_selector('button').click()

# Login
driver.find_element_by_css_selector('input').send_keys(username + Keys.TAB + password + Keys.ENTER)

In [ ]:
time.sleep(1)

        # Extract current hash tag name
        tag_name = tag

        # Extract all post links from 'explore tags' page
        post_links = []

        if scroll == 0:
            soup = BeautifulSoup(driver.page_source, "lxml")
            recent = soup.find_all('div', {'class': 'Nnq7C weEfm'})
            recent = recent[4:]
            for row in recent:
                for a in row.find_all('a', href=True):
                    post_links.append(a['href'])
        else:
            for times in range(scroll):
                driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                time.sleep(1)
            recent = driver.find_elements_by_class_name('KC1QD')
            for row in recent:
                recent = row.find_elements_by_tag_name('a')
            for a in recent:
                post_links.append(a.get_attribute('href').strip('https://www.instagram.com'))
            post_links = post_links[9:]

        # Keep link of only 30 most recent posts
        if scroll == 0:
            clean_post_links = [x for x in post_links if x.startswith('/p/')]
        else:
            clean_post_links = ['/p/' + x for x in post_links]

        del clean_post_links[30:]

        # Extract the info for each post of the tag
        for post_id in range(len(clean_post_links)):

            # Add tag name
            tag_name_list.append(tag)
            driver.get('https://www.instagram.com' + str(clean_post_links[post_id]))
            soup = BeautifulSoup(driver.page_source, "lxml")

            # Add post id
            post_ids_list.append(str(clean_post_links[post_id]))

            # Add likes, if any
            likes = soup.find('div', {'class': 'Nm9Fw'})
            if likes is None:
                likes_list.append('0')
            else:
                try:
                    likes_list.append(likes.find('span').text)
                except:
                    likes_list.append('1')

            # Add timestamp
            i = soup.find('time')
            if i is None:
                post_time_list.append(np.nan)
            else:
                post_time_list.append(i['datetime'])

driver.quit()

# Add hash tag info to data frame

tag_df = pd.DataFrame({'tags': tag_name_list, 'postid': post_ids_list, 'date': post_time_list, 'likes': likes_list})

# Data cleaning: removal of duplicates and incomplete data
tag_df['postid'] = tag_df['postid'].str.strip('/p/')
tag_df['date'] = tag_df['date'].astype('datetime64')
tag_df['likes'] = tag_df['likes'].astype('float64')
unique_tags_df = tag_df.drop_duplicates(subset='postid')
unique_tags_df_nona = unique_tags_df.dropna()

# upload to DB
driver = 'mysql+pymysql:'
user = 'root'
password = ''#password goes here
ip = '35.195.111.11'
database = 'Avocalypse'

connection_string = f'{driver}//{user}:{password}@{ip}/{database}'
engine = create_engine(connection_string)

unique_tags_df_nona.to_sql('insta_posts_notnull', con=engine)